In [16]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from jira import JIRA
import requests
class Jira(object):
    options = {
        'server':'https://aptude.atlassian.net'
    }
    client = None
    def __init__(self, **kwargs):
        self.__username = kwargs['username']
        self.__password = kwargs['password']
        try:
            self.client = JIRA(self.options, basic_auth=(self.__username, self.__password))
        except:
            raise JiraException('Could not connect to the API, invalid username or password!') from None        
    def getProjects(self, raw = False):
        Projects = pd.DataFrame()
        P = pd.DataFrame()
        for project in self.client.projects():
            if raw:
                P = Projects.append(project)
            else:
                a = Projects.append(pd.Series([project.key,project.name],index=['Project Key','Name']),ignore_index=True)
                P = P.append(a)
        return P

    def getIssues(self, maxResults = 10, raw = False, **kwargs):
        Issues = pd.DataFrame()
        I = pd.DataFrame()
        searchstring = ' '.join([( _ + "=" + kwargs[_]) if _ != 'condition' else kwargs[_] for _ in kwargs])
        for item in self.client.search_issues(searchstring, maxResults = maxResults):
            if raw:
                I = Issues.append(item)
            else:
                a = Issues.append(pd.Series([item.fields.assignee, item.fields.summary, item.fields.description, item.fields.status, item.fields.created],index = ['Assignee', 'Title', 'Description', 'Status', 'Start Date']),ignore_index=True)
                I = I.append(a)                  
        return I

In [17]:
print("enter Username")
#rkunchakuri@aptude.com
u=input()
print("\n")
print("enter Password")
#gXjUPCbVna6WRP9f9xvh986F
p=input()
print("\n")
MyJira = Jira(username= u, password=p)
print(MyJira.getProjects())
print("\n")
print("Enter the Project Key to get further details")
key = input()
print("\n")
print(MyJira.getIssues(project=key))

enter Username
rkunchakuri@aptude.com


enter Password
gXjUPCbVna6WRP9f9xvh986F


                Name Project Key
0             aptude          AP
0  Aptude Team Edgar         APT
0          Schneider   SCHNEIDER


Enter the Project Key to get further details
AP


      Assignee                                        Description  \
0  rkunchakuri  Keys needed in a word table\r\n\r\nUser Story ...   
0  rkunchakuri  Create one function by now:\r\n\r\n`get_open_t...   

                     Start Date       Status  \
0  2020-11-24T13:35:25.282-0600  In Progress   
0  2020-11-05T13:12:52.725-0600         Done   

                                               Title  
0  Drop the info get by tha Jira API into a word ...  
0    Create helpers to get information from JIRA API  
